# 生成数据集
（1）通过张量来进⾏数据存储和线性代数；  
（2）通过⾃动微分来计算梯度；

In [13]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

# 读取数据集
可以调⽤框架中现有的API来读取数据。我们将features和labels作为API的参数传递，并通过数据迭代器指定batch_size。此外，布尔值is_train表⽰是否希望数据迭代器对象在每个迭代周期内打乱数据。

In [14]:
def load_array(data_arrays, batch_size, is_train=True): #@save
    """构造⼀个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [15]:
#使⽤next从迭代器中获取第⼀项
next(iter(data_iter))

[tensor([[ 1.5003, -1.2882],
         [ 1.0363,  1.1068],
         [ 0.0471, -0.1982],
         [-0.0971,  0.8290],
         [ 0.5953, -0.5932],
         [ 0.4478,  1.1980],
         [-0.6015, -0.9830],
         [ 0.5716, -0.9630],
         [ 0.5563,  0.0554],
         [ 1.3279,  0.8224]]),
 tensor([[11.5886],
         [ 2.5153],
         [ 4.9774],
         [ 1.1870],
         [ 7.4302],
         [ 1.0137],
         [ 6.3257],
         [ 8.6073],
         [ 5.1254],
         [ 4.0579]])]

# 定义模型
对于标准深度学习模型，我们可以使⽤框架的预定义好的层。只需关注使⽤哪些层来构造模型，⽽不必关注层的实现细节。⾸先定义⼀个模型变量net，它是⼀个Sequential类的实例。Sequential类将多个层串联在⼀起。当给定输⼊数据时，Sequential实例将数据传⼊到第⼀层，然后将第⼀层的输出作为第⼆层的输⼊，以此类推。  
全连接层：每⼀个输⼊都通过矩阵-向量乘法得到它的每个输出；  
下面将将两个参数传递到nn.Linear中，第⼀个指定输⼊特征形状，即2，第⼆个指定输出特征形状，输出特征形状为单个标量，因此为1。

In [16]:
# nn是神经⽹络的缩写
from torch import nn
net = nn.Sequential(nn.Linear(2, 1))

# 初始化模型
初始化线性回归模型中的权重和偏置，指定每个权重参数应该从均值为0、标准差为0.01的正态分布中随机采样，偏置参数将初始化为零。  
正如我们在构造nn.Linear时指定输⼊和输出尺⼨⼀样，现在我们能直接访问参数以设定它们的初始值。我们通过net[0]选择⽹络中的第⼀个图层，然后使⽤weight.data和bias.data⽅法访问参数。我们还可以使⽤替换⽅法normal_和fill_来重写参数值。

In [17]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

# 定义损失函数
计算均⽅误差使⽤的是MSELoss类，也称为平⽅L2范数。默认情况下，它返回所有样本损失的平均值。

In [18]:
loss = nn.MSELoss()

# 定义优化函数
⼩批量随机梯度下降算法是⼀种优化神经⽹络的标准⼯具，PyTorch在optim模块中实现了该算法的许多变种。当我们实例化⼀个SGD实例时，我们要指定优化的参数（可通过net.parameters()从我们的模型中获得）以及优化算法所需的超参数字典。⼩批量随机梯度下降只需要设置lr值（学习率），这⾥设置为0.03。

In [19]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

# 开始训练
在简洁实现线性回归的过程中，不必单独分配参数、不必定义我们的损失函数，也不必⼿动实现⼩批量随机梯度下降。这就体现了高级API的优势。  
在每个迭代周期⾥，我们将完整遍历⼀次数据集（train_data），不停地从中获取⼀个⼩批量的输⼊和相应的标签。对于每⼀个⼩批量，我们会进⾏以下步骤:
* 通过调⽤net(X)⽣成预测并计算损失l（前向传播）。
* 通过进⾏反向传播来计算梯度。
* 通过调⽤优化器来更新模型参数。

In [20]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000178
epoch 2, loss 0.000104
epoch 3, loss 0.000105


In [21]:
#查看误差
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([-0.0008,  0.0004])
b的估计误差： tensor([0.0003])
